#  augmentation of grouped PNGs in folder, save origins and results in one new folder

In [3]:
import os
from PIL import Image, ImageEnhance
import numpy as np
import random

# Define augmentation functions
def flip_image(img, horizontal=True):
    """Flip the image horizontally or vertically."""
    if horizontal:
        return img.transpose(Image.FLIP_LEFT_RIGHT)
    else:
        return img.transpose(Image.FLIP_TOP_BOTTOM)

def rotate_image(img, angle=15):
    """Rotate the image by a specified angle."""
    return img.rotate(angle)

def zoom_image(img, zoom_factor=1.2):
    """Zoom into the image by a specified factor."""
    width, height = img.size
    x = int(width / zoom_factor)
    y = int(height / zoom_factor)
    img_cropped = img.crop(((width - x) // 2, (height - y) // 2, (width + x) // 2, (height + y) // 2))
    return img_cropped.resize((width, height), Image.LANCZOS)

def adjust_brightness(img, factor=1.5):
    """Adjust the brightness of the image."""
    enhancer = ImageEnhance.Brightness(img)
    return enhancer.enhance(factor)

def add_noise(img, noise_level=0.1):
    """Add random noise to the image."""
    img_array = np.array(img)
    noise = np.random.normal(0, noise_level * 255, img_array.shape).astype(np.uint8)
    img_noisy = np.clip(img_array + noise, 0, 255).astype(np.uint8)
    return Image.fromarray(img_noisy)

# Function to randomly apply one augmentation
def random_augmentation(img):
    augmentations = [
        flip_image,
        # rotate_image,
        zoom_image,
        adjust_brightness,
        add_noise
    ]
    # Randomly choose an augmentation method
    augmentation = random.choice(augmentations)
    
    # Apply with random parameters based on the function
    if augmentation == rotate_image:
        return augmentation(img, angle=random.choice([15, 30, 45]))
    elif augmentation == flip_image:
        return augmentation(img, horizontal=random.choice([True, False]))
    elif augmentation == zoom_image:
        return augmentation(img, zoom_factor=random.uniform(1.1, 1.5))
    elif augmentation == adjust_brightness:
        return augmentation(img, factor=random.uniform(0.5, 1.5))
    elif augmentation == add_noise:
        return augmentation(img, noise_level=random.uniform(0.002, 0.003))
    return img

# Main processing function
def process_images(input_folder):
    output_folder = os.path.join(input_folder, "augmented_images")
    os.makedirs(output_folder, exist_ok=True)

    # Loop over all images in the input folder
    for filename in os.listdir(input_folder):
        file_path = os.path.join(input_folder, filename)
        
        # Only process .png images
        if filename.lower().endswith('.png'):
            # Load the image
            img = Image.open(file_path)
            
            # Save the original image in the output folder
            original_path = os.path.join(output_folder, f"original_{filename}")
            img.save(original_path)
            
            # Apply a random augmentation
            img_augmented = random_augmentation(img)
            
            # Save the augmented image
            augmented_path = os.path.join(output_folder, f"augmented_{filename}")
            img_augmented.save(augmented_path)
            print(f"Saved original and augmented images for {filename}")

# Set the path to your folder containing PNG images
input_folder = './Data/fits_filtered9'  # Replace with the path to your folder
process_images(input_folder)


Saved original and augmented images for tic1.png
Saved original and augmented images for tic10.png
Saved original and augmented images for tic11.png
Saved original and augmented images for tic12.png
Saved original and augmented images for tic13.png
Saved original and augmented images for tic14.png
Saved original and augmented images for tic15.png
Saved original and augmented images for tic16.png
Saved original and augmented images for tic17.png
Saved original and augmented images for tic18.png
Saved original and augmented images for tic2.png
Saved original and augmented images for tic27.png
Saved original and augmented images for tic28.png
Saved original and augmented images for tic29.png
Saved original and augmented images for tic3.png
Saved original and augmented images for tic6.png
